In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

import gomoku, player_basic, player_mcts

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

# Playtest

In [12]:
game = gomoku.Gomoku(9)
model = player_mcts.net(9)

In [13]:
opt = keras.optimizers.Adam(learning_rate=1e-4)
policy_loss = keras.losses.BinaryCrossentropy(name='cross_entropy')
value_loss = keras.losses.MeanSquaredError(name='mse')

model.compile(optimizer=opt, 
              loss={'policy': policy_loss,
                    'value': value_loss})

In [14]:
recorder = player_mcts.GameRecorder('games_1.tfrecords')
p1 = player_mcts.ZeroPlayer('p1', +1, game, model, recorder)
p2 = player_mcts.ZeroPlayer('p2', -1, game, model, recorder)

In [15]:
pa, pb = p1, p2
while not game.finished:
    game.play(*pa.play(game))
    pa, pb = pb, pa

game.show()

black played (1, 5).
white played (4, 1).
game has ended. winner: white
   0 1 2 3 4 5 6 7 8
 0 ● ● . ● ● ○ . . ○ 
 1 ○ ○ ○ ○ ○ ○ . . . 
 2 ● . . . ● ○ . . . 
 3 ● . . . ● . ● . . 
 4 ● . . . ○ . ● . . 
 5 ● ● . ● ● ● ● ○ . 
 6 ○ ○ . . ● . ○ . . 
 7 ○ . ○ . . . . ○ . 
 8 . ○ . . ● . . ○ . 


In [16]:
recorder.close()

In [19]:
data = recorder.fetch()